In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load CSV normally (no header=1 needed based on previous analysis)
file_path = 'SupplyDemand.csv' # Adjust path if necessary
try:
    df = pd.read_csv(file_path)
    print(f"File '{file_path}' loaded successfully.")
    # Verify column names - should be 'Product ID', 'Attribute', 'Q2 95', ...
    # print("Columns loaded:", df.columns.tolist())
    # print(df.head(3)) # Uncomment to view the first few rows
except Exception as e:
    print(f"Error loading '{file_path}': {e}")
    df = pd.DataFrame() # Create empty DataFrame if loading fails

File 'SupplyDemand.csv' loaded successfully.


In [3]:
# Remove fully empty columns (if any) - less likely needed here
if not df.empty:
    df.dropna(axis=1, how='all', inplace=True)
    # print(f"Shape after removing empty columns: {df.shape}")

In [4]:
df_filtered_sd = pd.DataFrame() # Initialize empty
if not df.empty:
    # Expected column names from standard load
    id_col = 'Product ID'
    attr_col = 'Attribute'

    # *** Define the attributes needed for the optimization model ***
    # Check Definitions.csv or file content for exact names if unsure
    attributes_to_keep = [
        'EffectiveDemand',
        'Safety Stock Target',
        # Add the attribute representing initial inventory if available, e.g.:
        # 'Seedstock (Initial Inventory)',
        # 'On Hand' # (Need to confirm if 'On Hand' at Q2 95 is the initial state)
        'Total Projected Inventory Balance' # Keep this to extract initial inventory from the first period?
    ]

    if id_col in df.columns and attr_col in df.columns:
        # Filter for relevant products and attributes
        condition = (
            df[id_col].isin(['21A', '22B', '23C']) &
            df[attr_col].isin(attributes_to_keep)
        )
        df_filtered_sd = df.loc[condition].copy()
        print(f"Rows filtered by Product ID and selected Attributes. Shape: {df_filtered_sd.shape}")
        # print(df_filtered_sd[[id_col, attr_col]].head()) # Uncomment to view head
        # print("Unique Attributes kept:", df_filtered_sd[attr_col].unique()) # Verify filtering
    else:
        print(f"Error: Columns '{id_col}' or '{attr_col}' not found. Check file loading.")

Rows filtered by Product ID and selected Attributes. Shape: (9, 37)


In [5]:
if not df_filtered_sd.empty:
    df_filtered_sd.drop_duplicates(inplace=True)

In [6]:
if not df_filtered_sd.empty:
    df_filtered_sd.columns = df_filtered_sd.columns.str.strip().str.replace(" ", "_").str.replace(".", "_", regex=False).str.replace("-", "_")
    print("Column names cleaned.") # Uncomment for confirmation
    print(df_filtered_sd.columns.tolist()) # Uncomment to view cleaned names

Column names cleaned.
['Product_ID', 'Attribute', 'Q2_95', 'Q3_95', 'Q4_95', 'Q1_96', 'Q2_96', 'Q3_96', 'Q4_96', 'Q1_97', 'Q2_97', 'Q3_97', 'Q4_97', 'Q1_98', 'Q2_98', 'Q3_98', 'Q4_98', 'Q1_99', 'Q2_99', 'Q3_99', 'Q4_99', 'Q1_00', 'Q2_00', 'Q3_00', 'Q4_00', 'Q1_01', 'Q2_01', 'Q3_01', 'Q4_01', 'Q1_02', 'Q2_02', 'Q3_02', 'Q4_02', 'Q1_03', 'Q2_03', 'Q3_03', 'Q4_03']


In [7]:
if not df_filtered_sd.empty:
    # Values start from the THIRD column (index 2) onwards
    value_columns = df_filtered_sd.columns[2:]
    print(f"Attempting to convert {len(value_columns)} quarterly columns to numeric...")
    for col in value_columns:
        # Clean column name before conversion if not done in previous step
        clean_col_name = col # Already cleaned in previous cell
        df_filtered_sd[clean_col_name] = pd.to_numeric(df_filtered_sd[clean_col_name], errors='coerce')

    nan_count = df_filtered_sd[value_columns].isnull().sum().sum()
    if nan_count > 0:
        print(f"Warning: {nan_count} NaNs generated during numeric conversion.")
        # Decide how to handle missing Demand/SST (e.g., fill with 0?)
        # Example: df_filtered_sd[value_columns] = df_filtered_sd[value_columns].fillna(0)
    else:
        print("Numeric conversion completed without NaNs.")
    # print(df_filtered_sd.info()) # Uncomment to verify dtypes

Attempting to convert 35 quarterly columns to numeric...


In [8]:
df_long_sd = pd.DataFrame() # Initialize empty
if not df_filtered_sd.empty:
    try:
        # Use cleaned column names
        id_col_clean = df_filtered_sd.columns[0]     # e.g., Product_ID
        attr_col_clean = df_filtered_sd.columns[1]   # e.g., Attribute
        value_vars = df_filtered_sd.columns[2:]      # e.g., Q2_95, Q3_95,...

        # Rename identifier columns for final output
        df_to_melt = df_filtered_sd.rename(columns={
            id_col_clean: 'Product_ID',
            attr_col_clean: 'Attribute'
        })

        df_long_sd = pd.melt(
            df_to_melt,
            id_vars=['Product_ID', 'Attribute'],
            value_vars=value_vars,
            var_name='Quarter',       # Name for the column holding original Quarter names
            value_name='Value'        # Name for the column holding the corresponding value
        )
        print(f"Supply/Demand DataFrame reshaped to long format. Final shape: {df_long_sd.shape}")
        # print(df_long_sd.head()) # Uncomment to view result
    except Exception as e:
      print(f"Error during Supply/Demand melt: {e}")

Supply/Demand DataFrame reshaped to long format. Final shape: (315, 4)


In [9]:
output_file = "SupplyDemand_Long.csv"
if not df_long_sd.empty:
    df_long_sd.to_csv(output_file, index=False)
    print(f"Processed Supply/Demand data saved to '{output_file}'")
else:
    print("Final Supply/Demand DataFrame is empty, file not saved.")

Processed Supply/Demand data saved to 'SupplyDemand_Long.csv'
